[Date Functions in Standard SQL
](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions)

In [124]:
# Import Necessary Libraries

from google.cloud import bigquery
from google.cloud.bigquery import Dataset
from google.cloud import storage

In [125]:
# 
key_path = 'JSON_FILE_LOCATION'

# Project created as a owner
private_project = 'PROJECT_ID'

# Public project of big query
public_project = 'bigquery-public-data'

# Set storage client
storage_client = storage.Client.from_service_account_json(key_path)

# Set big query client
client = bigquery.Client.from_service_account_json(key_path, project=public_project)

In [127]:
dataset_id = 'stackoverflow'

# Get the public dataset
dataset_ref = client.dataset(dataset_id, project=public_project)

In [128]:
def printTableList(client, dataset_id):
    project = client.project
    dataset_ref = client.dataset(dataset_id, project = project)    
    tables = list(client.list_tables(dataset_ref))
    if tables:
        print('Tables in dataset {}:'.format(dataset_id))
        for table in tables: 
            print('\t{}'.format(table.table_id))
        found = True
    else:
        print('{} dataset does not contain any tables.'.format(dataset_id))
        found = False
    return found

In [129]:
printTableList(client, dataset_id)

Tables in dataset stackoverflow:
	badges
	comments
	post_history
	post_links
	posts_answers
	posts_moderator_nomination
	posts_orphaned_tag_wiki
	posts_privilege_wiki
	posts_questions
	posts_tag_wiki
	posts_tag_wiki_excerpt
	posts_wiki_placeholder
	stackoverflow_posts
	tags
	users
	votes


True

In [130]:
# Set the project as private to run queries
client = bigquery.Client.from_service_account_json(key_path, project=private_project)

### Sample Questions

#### 1. What percentage of questions have been answered over the years?

Use following table 

> `bigquery-public-data.stackoverflow.posts_questions`

Present data visually.

In [ ]:
sql =  '''SELECT
  EXTRACT(YEAR FROM creation_date) AS year
  , COUNT(*)
  , ROUND(COUNT(CASE WHEN answer_count > 0 THEN 1 ELSE NULL END) / COUNT(answer_count), 2) * 100 AS answered
FROM `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY year
ORDER BY year'''

query_job = client.query(sql)
df = query_job.to_dataframe()

In [ ]:
y = df.set_index('year')
y['answered'].plot();

In [ ]:
y['answered'].plot(kind='bar');

#### 2. What are the 10 ‘easiest’ gold badges to earn?

Use following tables:

> - `bigquery-public-data.stackoverflow.badges`
- `bigquery-public-data.stackoverflow.users`

- How many classes are there?
- Badges of which classes are easy to earn?

In [ ]:
sql = '''
SELECT 
  name
  , AVG(DATE_DIFF(DATE(`bigquery-public-data.stackoverflow.badges`.date), DATE(`bigquery-public-data.stackoverflow.users`.creation_date), DAY)) as days_diff
 FROM `bigquery-public-data.stackoverflow.badges`
 JOIN `bigquery-public-data.stackoverflow.users` ON `bigquery-public-data.stackoverflow.users`.id = `bigquery-public-data.stackoverflow.badges`.user_id
 WHERE class = 1 AND tag_based = False
 GROUP BY name
 ORDER BY days_diff ASC
 LIMIT 10
'''

query_job = client.query(sql)
df = query_job.to_dataframe()

In [ ]:
y = df.set_index('name')
y.sort_values(by='days_diff', ascending=False).plot(kind='barh');

#### 3. Which day of the week has most questions answered within an hour?

Use following tables:

> - `bigquery-public-data.stackoverflow.posts_answers`
- `bigquery-public-data.stackoverflow.posts_questions`

In [ ]:
sql = '''
    SELECT
      FORMAT_DATE('%A', DATE(`bigquery-public-data.stackoverflow.posts_answers`.creation_date)) AS day_of_week,
      EXTRACT(DAYOFWEEK FROM DATE(`bigquery-public-data.stackoverflow.posts_answers`.creation_date)) AS day_of_week_,
      COUNT(TIMESTAMP_DIFF(`bigquery-public-data.stackoverflow.posts_answers`.creation_date, `bigquery-public-data.stackoverflow.posts_questions`.creation_date, MINUTE)) AS time_diff
    FROM `bigquery-public-data.stackoverflow.posts_answers` 
    JOIN `bigquery-public-data.stackoverflow.posts_questions` ON `bigquery-public-data.stackoverflow.posts_questions`.id =  `bigquery-public-data.stackoverflow.posts_answers`.parent_id
    WHERE TIMESTAMP_DIFF(`bigquery-public-data.stackoverflow.posts_answers`.creation_date, `bigquery-public-data.stackoverflow.posts_questions`.creation_date, MINUTE) <= 60
    GROUP BY day_of_week, day_of_week_
    ORDER BY day_of_week_
    LIMIT 1000
'''
query_job = client.query(sql)
df = query_job.to_dataframe()

In [ ]:
df.head()

In [ ]:
df.drop(labels=['day_of_week_'], inplace=True, axis=1)
y = df.set_index('day_of_week')
y.plot();

#### 4. Does adding a “?” actually help user to get answers?

Sara Robinson claimed that only 21% questions end with a question mark(?).

<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Did you know...only 21% of <a href="https://twitter.com/StackOverflow?ref_src=twsrc%5Etfw">@StackOverflow</a> questions end with a question mark? <a href="https://t.co/b939b3xwU0">https://t.co/b939b3xwU0</a> <a href="https://t.co/LfVnjrPPJw">pic.twitter.com/LfVnjrPPJw</a></p>&mdash; Sara Robinson (@SRobTweets) <a href="https://twitter.com/SRobTweets/status/810892697116545024?ref_src=twsrc%5Etfw">December 19, 2016</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

- Verify her claim.

- Do you think adding question mark(?) at the end of question help user to get answer?

In [ ]:
sql = '''
SELECT
  EXTRACT(YEAR
  FROM
    creation_date) AS year,
  IF(title LIKE '%?',
    'ends with ?',
    'does not end with ?') AS ends_with_question_mark,
  ROUND(COUNT(accepted_answer_id ) * 100 / COUNT(*), 2) AS answered
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  year,
  ends_with_question_mark
ORDER BY
  year,
  ends_with_question_mark
'''
query_job = client.query(sql)
df = query_job.to_dataframe()

In [ ]:
x = df[['year','ends_with_question_mark','answered' ]]
y = x.set_index('year')
z = y.groupby(by=['year', 'ends_with_question_mark']).max()

In [ ]:
ax = z.unstack().plot(kind='bar', label='x')
ax.legend(["does not end with ?", "ends with ?"]);

### What's Next?

This is only a start. Using tables of stackoverflow create your own question and answer them using data!

For inspiration try to answer [these](https://data.stackexchange.com/stackoverflow/queries) queries.